In [45]:
import numpy as np
import json
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import torch
import zipfile
import bz2
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [3]:
#https://ai.stanford.edu/~amaas/data/sentiment/

In [10]:
data_main_path = "../data/movie_reviews/aclImdb/"

In [16]:
train_pos_files = [f for f in listdir(data_main_path + "train/pos/") if isfile(join(data_main_path + "train/pos/", f))]
train_neg_files = [f for f in listdir(data_main_path + "train/neg/") if isfile(join(data_main_path + "train/neg/", f))]
test_pos_files = [f for f in listdir(data_main_path + "test/pos/") if isfile(join(data_main_path + "test/pos/", f))]
test_neg_files = [f for f in listdir(data_main_path + "test/neg/") if isfile(join(data_main_path + "test/neg/", f))]
print(train_pos_files[0:10])

['4715_9.txt', '12390_8.txt', '8329_7.txt', '9063_8.txt', '3092_10.txt', '9865_8.txt', '6639_10.txt', '10460_10.txt', '10331_10.txt', '11606_10.txt']


In [18]:
train_pos_all = []
for i in train_pos_files:
    with open(data_main_path + "train/pos/" + i) as f:
        train_pos_all.append(f.read())

In [22]:
train_neg_all = []
for i in train_neg_files:
    with open(data_main_path + "train/neg/" + i) as f:
        train_neg_all.append(f.read())

In [23]:
test_pos_all = []
for i in test_pos_files:
    with open(data_main_path + "test/pos/" + i) as f:
        test_pos_all.append(f.read())

In [24]:
test_neg_all = []
for i in test_neg_files:
    with open(data_main_path + "test/neg/" + i) as f:
        test_neg_all.append(f.read())

# check length and combine all data

In [25]:
len(train_pos_all), len(train_neg_all), len(test_pos_all), len(test_neg_all)

(12500, 12500, 12500, 12500)

In [30]:
all_data = train_pos_all + train_neg_all + test_pos_all + test_neg_all
all_label = [2 for _ in range(len(train_pos_all))] + [1 for _ in range(len(train_neg_all))] + [2 for _ in range(len(test_pos_all))] + [1 for _ in range(len(test_neg_all))] 

In [31]:
len(all_data), len(all_label)

(50000, 50000)

In [34]:
X_train_val, X_test, y_train_val, y_test = train_test_split(all_data, all_label, test_size=0.33, random_state=7)

In [36]:
len(X_train_val), len(y_train_val)

(33500, 33500)

# output data dictionary

In [49]:
data_dict = {"text":all_data, "label":all_label}

In [ ]:
with open("../data/movie_reviews/all_dict.txt","r") as f:
    json.dump(data_dict, f, indent=4)

In [41]:
print(sum([x==1 for x in y_train_val[:2000]])/len(y_train_val[:2000])) # data is balanced
print(sum([x==1 for x in y_test[:2000]])/len(y_test[:2000])) # data is balanced

0.501
0.502


In [40]:
def tokenize_encode_sentences(tokenizer, model, input_sentences, output_path):
    output = np.zeros([len(input_sentences), 768])
    for i, x in enumerate(input_sentences): 
        output[i] = tokenize_encode_sentences_sample(tokenizer, model, [x])
    np.save(output_path, output)
    return output

def tokenize_encode_sentences_sample(tokenizer, model, input_sentences):
    encoded_input = tokenizer(input_sentences, return_tensors = 'pt', truncation = True, padding = True)
    output = model(**encoded_input)[0][:,0,:].detach().numpy()
    return output

In [46]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [55]:
bert_train = tokenize_encode_sentences(tokenizer, model, list(X_train_val[:2000]), "../data/all_bert" + "encoded_aclimbd_train_2000")
# bert_test = tokenize_encode_sentences(tokenizer, model, list(X_test[:2000]), "../data/all_bert" + "encoded_aclimbd_test_2000")

In [52]:
aclimbd_array = np.array([bert_train, y_train_val[:2000], "aclimbd"])
np.save(data_main_path + "movie_review.npy", aclimbd_array)

<ipython-input-52-4b79dff5ed0e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  aclimbd_array = np.array([bert_train, y_train_val[:2000], "aclimbd"])


In [54]:
aclimbd_array[0]

array([[-0.05888786, -0.01375623, -0.05178594, ..., -0.17930274,
         0.66051066,  0.62604874],
       [-0.21833044, -0.30038214, -0.2517412 , ..., -0.05173419,
         0.76385438,  0.30639154],
       [ 0.36435646, -0.28725877, -0.15936121, ..., -0.05094235,
         0.58515561,  0.3106201 ],
       ...,
       [-0.00734697, -0.5028249 ,  0.2238474 , ..., -0.03800585,
         0.57313204,  0.51405483],
       [-0.25712967, -0.34837094,  0.05731477, ..., -0.29740858,
         0.46573183,  0.59045959],
       [ 0.21649717, -0.1659523 ,  0.53854537, ..., -0.06250568,
         0.44901812,  0.16673516]])